In [97]:
import tensorflow as tf
import time
from PIL import Image
import os
import numpy as np
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import tensor_shape
from tensorflow.python.platform import gfile
from tensorflow.python.util import compat
import argparse
from datetime import datetime
import hashlib
import random
import re
import sys
import tarfile
from six.moves import urllib

from matplotlib.pyplot import imshow
from matplotlib.pyplot import imsave

import math
import random



In [64]:
input_width = 299
input_height = 299
input_depth = 3
input_mean = 128
input_std = 128

In [142]:
def add_input_distortions(random_crop, random_scale, random_brightness):
    jpeg_data = tf.placeholder(tf.string, name='JPGInput')
    decode_image = tf.image.decode_jpeg(jpeg_data, channels=3)
    decode_image_as_float = tf.cast(decode_image, dtype=tf.float32)
    decode_image_4d = tf.expand_dims(decode_image_as_float, 0)
    margin_scale = 1.0 + (random_crop / 100.0)
    resize_scale = 1.0 + (random_scale / 100.0)
    
    margin_scale_value = tf.constant(margin_scale)
    resize_scale_value = tf.random_uniform(tensor_shape.scalar(), minval=1.0, maxval=resize_scale)
    
    scale_value = tf.multiply(margin_scale_value, resize_scale_value)
    precrop_width = tf.multiply(scale_value, input_width)
    precrop_height = tf.multiply(scale_value, input_height)
    precrop_shape = tf.stack([precrop_height, precrop_width])
    precrop_shape_as_int = tf.cast(precrop_shape, dtype=tf.int32)
    precropped_image = tf.image.resize_bilinear(decode_image_4d, precrop_shape_as_int)
    #precropped_image = tf.image.resize_bilinear(decode_image_as_float, precrop_shape_as_int)
    precropped_image_3d = tf.squeeze(precropped_image, squeeze_dims=[0])
    cropped_image = tf.random_crop(precropped_image_3d, [input_height, input_width, input_depth])
    
    flip = random.randrange(0,2)
    if flip == 1:
        flipped_image = tf.image.random_flip_left_right(cropped_image)
    else:
        flipped_image = cropped_image
    
    brightness_min = 1.0 - (random_brightness / 100.0)
    brightness_max = 1.0 + (random_brightness / 100.0)
    brightness_value = tf.random_uniform(tensor_shape.scalar(), minval=brightness_min, maxval=brightness_max)
    brightened_image = tf.multiply(flipped_image, brightness_value)
    
    offset_image = tf.subtract(brightened_image, input_mean)
    mul_image = tf.multiply(offset_image, 1.0 / input_std)
    distort_result = tf.expand_dims(mul_image, 0, name='DistortResult')
    return jpeg_data, distort_result
    
    

In [138]:
def get_random_distorted_bottlenecks(distorted_jpeg, distorted_image):
    jpeg_data = gfile.FastGFile('/root/ss/image/13.jpg', 'rb').read()
    distorted_image_data= sess.run(distorted_image, {distorted_jpeg:jpeg_data})
    
    return distorted_image_data

In [144]:
step=50
for i in range(step):
    distorted_jpeg_data_tensor, distorted_image_tensor = add_input_distortions(20, 50, 70)
    result = get_random_distorted_bottlenecks(distorted_jpeg_data_tensor, distorted_image_tensor)
    imsave('/root/temp/'+str(i)+'.jpg', np.asarray(np.squeeze(result)))
    #imsave('/root/temp/'+str(i)+'.jpg', np.asarray(result.reshape(299,299,3)))
    print ("Step : {}".format(i))

Step : 0
Step : 1
Step : 2
Step : 3
Step : 4
Step : 5
Step : 6
Step : 7
Step : 8
Step : 9
Step : 10
Step : 11
Step : 12
Step : 13
Step : 14
Step : 15
Step : 16
Step : 17
Step : 18
Step : 19
Step : 20
Step : 21
Step : 22
Step : 23
Step : 24
Step : 25
Step : 26
Step : 27
Step : 28
Step : 29
Step : 30
Step : 31
Step : 32
Step : 33
Step : 34
Step : 35
Step : 36
Step : 37
Step : 38
Step : 39
Step : 40
Step : 41
Step : 42
Step : 43
Step : 44
Step : 45
Step : 46
Step : 47
Step : 48
Step : 49
